For the code in this noteook to run you will need to install the following packages:
    pip istall tensorflow
    pip install tensorflow_datasets

In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [3]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

2022-02-16 12:20:27.983500: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dataset imdb_reviews downloaded and prepared to /Users/xbbncc8/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2022-02-16 12:21:33.752508: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [4]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


2022-02-16 12:24:44.309742: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'"Most of us at least inhabit two worlds , the real world where we are at the mercy of circumstances and the world within ,the unconscious ,a safe place where we can escape ..." With those words ,Mr .Gone introduces inside the world of "The Maxx" a fascinating world where the fantasy and the reality are combined . Inspired in the comic books of Sam Kieth, "The Maxx " is very faithful to the material in what it was inspired , not only in the story but also in the graphic style ,that look like the pages of the comic ,giving this show a surreal and unique appearance . But also ,the story it\'s interesting and entertaining .At moments it could turn too weird ,but when you got inside it ,it\'s hard to get out of it . The story and the characters are wonderfully developed . The music goes perfectly with the style of the show and give it the proper atmosphere . Unfortunately , like many good animated shows ,this one was short -lived . "The Maxx" is a must see . It \'s one of my all 

2022-02-16 13:37:21.084779: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [6]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [7]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 88,   5, 170, ...,   0,   0,   0],
       [ 10, 208,  11, ...,   0,   0,   0],
       [255,  10, 153, ...,   0,   0,   0]])

In [8]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'"Most of us at least inhabit two worlds , the real world where we are at the mercy of circumstances and the world within ,the unconscious ,a safe place where we can escape ..." With those words ,Mr .Gone introduces inside the world of "The Maxx" a fascinating world where the fantasy and the reality are combined . Inspired in the comic books of Sam Kieth, "The Maxx " is very faithful to the material in what it was inspired , not only in the story but also in the graphic style ,that look like the pages of the comic ,giving this show a surreal and unique appearance . But also ,the story it\'s interesting and entertaining .At moments it could turn too weird ,but when you got inside it ,it\'s hard to get out of it . The story and the characters are wonderfully developed . The music goes perfectly with the style of the show and give it the proper atmosphere . Unfortunately , like many good animated shows ,this one was short -lived . "The Maxx" is a must see . It \'s one of my al